In [1]:
%load_ext autoreload
%autoreload 2

Welcome to `deferential_expression`. The purpose of this package is to give Python users easy broad access to differential expression libraries in R.

Using R-based libraries from Python can have a significant conversion overhead, depending on the size of this matrix. We solve this problem by converting expression to R once using `rpy2`, and providing access to R-based differential expression packages for these matrices.

Internally, we use our class `RMatrixAdapter`, to provide `numpy`-like behaviour for an R-matrix. Users, however, never needs to seeds to see `RMatrixAdapter`, storing their expression data in a BiocPy `SummarizedExperiment` and using a simple one-off conversion.

## Installation
For installation there are two main ways: (1) conda-based and (2) using System-installed R.

In the first case, you may use the provided `environment.yaml` file. This will install all required dependencies, including Python, R, `rpy2` and the R-packages, within a conda environment.

If you don´t want to use a conda-installed R, a system-installed R installation may be used. R needs to be added to `PATH`, and `rpy2` installed. `deferential_expression` can then be installed using 
`"deferential_expression @ git+https://github.com/MaximilianNuber/deferential_expression.git@main"`.
As for the R dependencies, using R functionalities for the first time will trigger a check, and missing dependencies will be installed automatically.

Alternatively, the repository can be cloned and installed with
```
pip install -e .
# (optional) 
# pip install -e ".[docs]"
```

## Quickstart: recount3 ➜ edgeR (all from Python)

But let´s get to an example of how this package can be used.

The snippet below runs R code from Python to download SRP149978 with recount3, extracts raw counts + sample metadata, builds an R-backed container, and runs an edgeR quasi-likelihood (QL) pipeline.

In [3]:
%%capture
# Setting up the data and converting from R.

import os
os.environ["R_HOME"] = "/Users/maximiliannuber/miniconda3/envs/def_exp_test_env/lib/R"

import pandas as pd
from bioc2ri.lazy_r_env import r  # lazy rpy2 env, see "bioc2ri"
import deferential_expression as de

# --- 1) Use recount3 in R to pull a moderate-sized project and make a 'condition' factor
r.ro.r("""

suppressMessages(library(recount3))
suppressMessages(library(stringr))

human_projects <- available_projects()
my_project <- subset(human_projects, project == "SRP149978" & project_type == "data_sources")
rse_gene <- create_rse(my_project)

rse_gene$condition <- rse_gene$sra.experiment_title |>
  stringr::str_split(":") |> sapply(function(x) x[2]) |>
  stringr::str_split(";") |> sapply(function(x) x[1]) |>
  stringr::str_split("_") |> sapply(function(x) x[1]) |>
  stringr::str_replace_all(" ", "") |>
  as.factor() |> relevel("Control")
""")

# --- 2) Pull counts (R matrix, stays R-backed) and colData (to pandas)
counts   = r.r2py(r.ro.r("assay(rse_gene, 'raw_counts')"))
coldata_df = r.r2py(r.ro.r("as.data.frame(colData(rse_gene))"))
rowdata_df = r.r2py(r.ro.r("as.data.frame(rowData(rse_gene))"))

We use the `recount3` Bioconductor package to get data from R. We have a `numpy` RNA-seq count matrix, sample-wise meta data, and feature wise meta data. 

Gene expression matrices in Bioconductor are stored in objects like `SummarizedExperiment` in the format `(genes x samples)`.

First, we store our data in `SummarizedExperiment` Python-only:

In [4]:
from summarizedexperiment import SummarizedExperiment

se = SummarizedExperiment(
    assays = {"counts": counts},
    column_data = coldata_df,
    row_data = rowdata_df
)

A common bottleneck, especially for large expression matrices, is the conversion from Python to R. We want to perform this conversion only once. In the following snippet, we construct an `RESummarizedExperiment`, which converts matrices in `assays` to R-matrices. Otherwise, it behaves like a BiocPy `SummarizedExperiment`.

In [5]:
rse = de.RESummarizedExperiment.from_summarized_experiment(se)

# take a look at the assay:
type(rse.assay_r("counts"))

rpy2.robjects.vectors.FloatMatrix

As you can see, the numpy array from SummarizedExperiment has been converted to an R-matrix.

Dedicated functions can use this R-matrix and enrich the `RESummarizedExperiment` with further assays from R libraries. Let´s test the `edgeR`-workflow:

In [6]:
# edgeR-preprocessing:

keep = de.edger.filter_by_expr(rse, group = rse.col_data["condition"])
print(
    pd.Series(keep).value_counts()
)

rse = rse[keep, :]

rse = de.edger.calc_norm_factors(rse)

rse.col_data.to_pandas()["norm.factors"].describe()

True     32222
False    31634
Name: count, dtype: int64


count    92.000000
mean      1.004803
std       0.097194
min       0.730631
25%       0.934352
50%       1.011183
75%       1.075588
max       1.211238
Name: norm.factors, dtype: float64

A small explanation of what happened: `de.edger.filter_by_expr` calls the R function `edgeR::filterByExpr` on the specified assay, by default "counts", and the output is automatically converted into a numpy-array and returned. Using this boolean array, we can simply subset an ´RESummarizedExperiment`. 

`de.edger.calc_norm_factors` calls `edger::calcNormFactors`, and adds the "norm.factors" column to the column data. All arguments can be specified and are forwarded to the R-function.


Next up, we want to use the edgeR model. First, we need to construct a design matrix. 
We do this in Python with current tools, we recommend `formulaic_contrasts`

In [7]:
from formulaic_contrasts import FormulaicContrasts

fc = FormulaicContrasts(rse.col_data.to_pandas(), "~condition")
design = fc.design_matrix

In [8]:
fit = de.edger.glm_ql_fit(rse, design = design)

The design matrix is a pandas dataframe and can be treated as such. We pass the `RESummarizedExperiment` and design to the fit-function, and get a Python-object containing the edgeR fit-object.

This fit-object can be passed to the test function:

In [9]:
res = de.edger.glm_ql_ftest(fit, coef = 2)

In [10]:
res

,logFC,logCPM,F,PValue,FDR
ENSG00000223972.5,0.149352,30.142036,0.538628,0.464784,0.661409
ENSG00000278267.1,0.664313,26.728468,9.757847,0.002373,0.032758
ENSG00000227232.5,0.265684,33.449910,9.872641,0.002229,0.031747
ENSG00000243485.5,-0.038612,24.673589,0.013832,0.906534,0.952470
ENSG00000239945.1,0.014600,28.045229,0.005973,0.938558,0.969457
...,...,...,...,...,...
ENSG00000279274.2,-0.218581,26.507425,2.391306,0.125375,0.307094
ENSG00000228786.5,-0.080047,25.265041,0.069426,0.792641,0.887383
ENSG00000229238.3,-0.555580,24.248539,2.084508,0.150467,0.341842
ENSG00000231514.1,-0.225417,26.020608,0.937149,0.335408,0.546330


In [11]:
rse.row_data.to_pandas()

,source,type,bp_length,phase,gene_id,gene_type,gene_name,level,havana_gene,tag,rownames
ENSG00000223972.5,HAVANA,gene,1735.0,-2147483648,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,OTTHUMG00000000961.2,NA_character_,ENSG00000223972.5
ENSG00000278267.1,ENSEMBL,gene,68.0,-2147483648,ENSG00000278267.1,miRNA,MIR6859-1,3,NA_character_,NA_character_,ENSG00000278267.1
ENSG00000227232.5,HAVANA,gene,1351.0,-2147483648,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,2,OTTHUMG00000000958.1,NA_character_,ENSG00000227232.5
ENSG00000243485.5,HAVANA,gene,1021.0,-2147483648,ENSG00000243485.5,lincRNA,MIR1302-2HG,2,OTTHUMG00000000959.2,ncRNA_host,ENSG00000243485.5
ENSG00000239945.1,HAVANA,gene,1319.0,-2147483648,ENSG00000239945.1,lincRNA,RP11-34P13.8,2,OTTHUMG00000001097.2,overlapping_locus,ENSG00000239945.1
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000279274.2,HAVANA,gene,75.0,-2147483648,ENSG00000279274.2,processed_pseudogene,RP11-533E23.2,2,OTTHUMG00000192107.1,NA_character_,ENSG00000279274.2
ENSG00000228786.5,HAVANA,gene,1192.0,-2147483648,ENSG00000228786.5,lincRNA,LINC00266-4P,2,OTTHUMG00000045201.1,overlapping_locus,ENSG00000228786.5
ENSG00000229238.3,HAVANA,gene,1671.0,-2147483648,ENSG00000229238.3,unprocessed_pseudogene,PPP1R12BP1,2,OTTHUMG00000036764.2,NA_character_,ENSG00000229238.3
ENSG00000231514.1,HAVANA,gene,640.0,-2147483648,ENSG00000231514.1,processed_pseudogene,FAM58CP,1,OTTHUMG00000036813.1,overlapping_locus,ENSG00000231514.1


In [12]:
rse = rse.set_row_names(rse.row_data["gene_name"])

NameError: name 'set_row_names' is not defined

In [12]:
from bioc2ri.rnames import get_rownames

In [20]:
rse.set_row_names(rse.row_data["gene_name"])

NameError: name 'set_row_names' is not defined